# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#pull csv from weatherpy
output_data_file = "data/cities.csv"
weather = pd.read_csv(output_data_file)
weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Dikson,100,RU,1619284545,85,73.5069,80.5464,13.96,18.45
1,Khandyga,56,RU,1619284772,99,62.6667,135.6000,-1.79,2.66
2,Escanaba,75,US,1619284772,58,45.7453,-87.0646,51.80,9.22
3,Ahipara,76,NZ,1619284772,85,-35.1667,173.1667,59.23,8.70
4,Viedma,88,AR,1619284732,54,-40.8135,-62.9967,62.73,18.68
...,...,...,...,...,...,...,...,...,...
542,Kempen,0,DE,1619284920,44,51.3667,6.4167,55.40,11.50
543,Ayaviri,27,PE,1619284920,38,-14.8864,-70.5889,57.65,12.82
544,Williams Lake,75,CA,1619284800,33,52.1415,-122.1445,46.99,5.75
545,Tabas,98,IR,1619284920,12,33.5959,56.9244,85.68,9.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
locations = weather[["Lat", "Lng"]]

humidity = weather["Humidity"].astype(float)

fig = gmaps.figure()
        
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=100,
                                   point_radius=3)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:

prefered = weather.loc[(weather["Wind Speed"] <= 10) & (weather["Cloudiness"] == 0) & \
                                   (weather["Max Temp"] >= 20) & (weather["Max Temp"] <= 50)].dropna()


prefered

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
17,Shuangyang,0,CN,1619284775,51,43.5242,125.6736,33.80,4.47
184,Nikolskoye,0,RU,1619284829,60,59.7035,30.7861,42.01,2.24
362,Maniitsoq,0,GL,1619284688,71,65.4167,-52.9000,38.10,0.76
482,Plastun,0,RU,1619284905,53,44.7561,136.2939,37.99,6.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Create a hotel_df
hotel_df = pd.DataFrame(prefered, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df=hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,Shuangyang,CN,43.5242,125.6736,
184,Nikolskoye,RU,59.7035,30.7861,
362,Maniitsoq,GL,65.4167,-52.9000,
482,Plastun,RU,44.7561,136.2939,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [44]:


for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f" City:{city_name} First Hotel: {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No hotel available.")
        
    print("***********")



Retrieving Results for Index 17: Shuangyang.
 City:Shuangyang First Hotel: Chenyu Hotel.
***********
Retrieving Results for Index 362: Maniitsoq.
 City:Maniitsoq First Hotel: Hotel Maniitsoq.
***********
Retrieving Results for Index 482: Plastun.
 City:Plastun First Hotel: Gostinitsa Rossiyanochka.
***********


In [31]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df=hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
17,Shuangyang,CN,43.5242,125.6736,Chenyu Hotel
362,Maniitsoq,GL,65.4167,-52.9000,Hotel Maniitsoq
482,Plastun,RU,44.7561,136.2939,Gostinitsa Rossiyanochka


In [33]:
hotel_name = []
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address = requests.get(base_url, params=params)
    address = address.json()
    try:

        hotel_name.append(address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))